In [4]:
from collections import defaultdict
import itertools
import numpy as np

In [14]:
SATURNIN_SBOX = [0x6, 0x9, 0x0, 0xE, 0x1, 0x5, 0x4, 0x7, 0xB, 0xD, 0x8, 0xF, 0x3, 0x2, 0xC, 0xA]

# just using s box the plainest of the plain attacks

In [ ]:
# Saturnin S-box (4-bit)

SBOX = SATURNIN_SBOX
# Inverse S-box
INV_SBOX = [0] * 16
for i in range(16):
    INV_SBOX[SBOX[i]] = i

def apply_sbox(x):
    """Apply S-box"""
    return SBOX[x]

def apply_inv_sbox(x):
    """Apply inverse S-box"""
    return INV_SBOX[x]

def apply_sbox_twice(x):
    """Apply S-box two times"""
    return SBOX[SBOX[x]]

def apply_inv_sbox_twice(x):
    """Apply inverse S-box two times"""
    return INV_SBOX[INV_SBOX[x]]

def boomerang_attack(alpha, delta, num_trials=256):
    """
    Test boomerang attack for given input difference alpha and output difference delta
    Returns the probability that we get alpha back and list of valid quartets
    """
    success_count = 0
    valid_quartets = []
    
    for p1 in range(16):  # Try all possible 4-bit inputs
        # Create pair with difference alpha
        p2 = p1 ^ alpha
        
        # Apply S-box twice (encryption)
        c1 = apply_sbox_twice(p1)
        c2 = apply_sbox_twice(p2)
        
        # Apply output difference delta
        c3 = c1 ^ delta
        c4 = c2 ^ delta
        
        # Apply inverse S-box twice (decryption)
        p3 = apply_inv_sbox_twice(c3)
        p4 = apply_inv_sbox_twice(c4)
        
        # Check if we got alpha back
        if (p3 ^ p4) == alpha:
            success_count += 1
            valid_quartets.append({
                'p1': p1, 'p2': p2, 'p3': p3, 'p4': p4,
                'c1': c1, 'c2': c2, 'c3': c3, 'c4': c4
            })
    
    probability = success_count / 16
    return probability, success_count, valid_quartets

# Test all possible difference pairs
print("Boomerang Attack on Saturnin S-box (applied twice)")
print("=" * 60)
print(f"{'Alpha (α)':>10} {'Delta (δ)':>10} {'Successes':>12} {'Probability':>12}")
print("-" * 60)

good_pairs = []

for alpha in range(1, 16):  # Skip 0 (no difference)
    for delta in range(1, 16):
        prob, count, quartets = boomerang_attack(alpha, delta)
        
        # Only print pairs with probability > random (1/16 = 0.0625)
        if prob > 0.0625:
            print(f"{alpha:>10} {delta:>10} {count:>12}/16 {prob:>12.4f}")
            good_pairs.append((alpha, delta, prob, quartets))

print("\n" + "=" * 60)
print(f"Found {len(good_pairs)} good boomerang pairs (prob > 1/16)")

if good_pairs:
    print("\nBest pairs:")
    good_pairs.sort(key=lambda x: x[2], reverse=True)
    for alpha, delta, prob, _ in good_pairs[:5]:
        print(f"  α={alpha:2}, δ={delta:2}: probability = {prob:.4f}")
    
    # Show detailed quartets for the best pair
    if good_pairs:
        print("\n" + "=" * 60)
        best_alpha, best_delta, best_prob, best_quartets = good_pairs[0]
        print(f"Detailed view of BEST pair: α={best_alpha}, δ={best_delta} (prob={best_prob:.4f})")
        print("=" * 60)
        print(f"{'P1':>4} {'P2':>4} {'C1':>4} {'C2':>4} | {'C3':>4} {'C4':>4} {'P3':>4} {'P4':>4} | P3⊕P4")
        print("-" * 60)
        for q in best_quartets:
            print(f"{q['p1']:>4} {q['p2']:>4} {q['c1']:>4} {q['c2']:>4} | "
                  f"{q['c3']:>4} {q['c4']:>4} {q['p3']:>4} {q['p4']:>4} | "
                  f"{q['p3']^q['p4']:>4} (α={best_alpha})")

Boomerang Attack on Saturnin S-box (applied twice)
 Alpha (α)  Delta (δ)    Successes  Probability
------------------------------------------------------------
         1          1            2/16       0.1250
         1          6            2/16       0.1250
         1          9            2/16       0.1250
         1         10            2/16       0.1250
         1         11            2/16       0.1250
         1         12            2/16       0.1250
         1         13            2/16       0.1250
         1         14            2/16       0.1250
         2          1           10/16       0.6250
         2          2            8/16       0.5000
         2          3            8/16       0.5000
         2          4            2/16       0.1250
         2          8            6/16       0.3750
         2          9            4/16       0.2500
         2         10            4/16       0.2500
         2         11            4/16       0.2500
         2         13   

# incompatibility in saturnin 2 round

In [20]:
#!/usr/bin/env python3
"""
Saturnin S-box Incompatibility Finder
Searches for (Δ, β, ∇) where DDT and BCT are nonzero but BDT is zero
"""

def compute_ddt(sbox):
    """Compute Differential Distribution Table"""
    n = len(sbox)
    ddt = [[0] * n for _ in range(n)]
    
    for x1 in range(n):
        for x2 in range(n):
            dx = x1 ^ x2
            dy = sbox[x1] ^ sbox[x2]
            ddt[dx][dy] += 1
    
    return ddt

def compute_bct(sbox):
    """Compute Boomerang Connectivity Table"""
    n = len(sbox)
    bct = [[0] * n for _ in range(n)]
    
    for dx in range(n):
        for dy in range(n):
            solutions = set()
            
            for x in range(n):
                x_prime = x ^ dx
                y = sbox[x]
                y_prime = sbox[x_prime]
                
                # Find z such that S(y ⊕ z) ⊕ S(y' ⊕ z) = dy
                for z in range(n):
                    val1 = sbox[y ^ z]
                    val2 = sbox[y_prime ^ z]
                    if (val1 ^ val2) == dy:
                        solutions.add(z)
            
            bct[dx][dy] = len(solutions)
    
    return bct

def compute_bdt(sbox):
    """Compute Boomerang Difference Table (3D table)"""
    n = len(sbox)
    bdt = [[[0] * n for _ in range(n)] for _ in range(n)]
    
    for dx in range(n):
        for beta in range(n):
            for dy in range(n):
                count = 0
                
                # Check all possible x values
                for x in range(n):
                    x_prime = x ^ dx
                    y = sbox[x]
                    y_prime = sbox[x_prime]
                    
                    # Check if output difference is beta
                    if (y ^ y_prime) != beta:
                        continue
                    
                    # Check if there exists z such that S(y ⊕ z) ⊕ S(y' ⊕ z) = dy
                    found = False
                    for z in range(n):
                        val1 = sbox[y ^ z]
                        val2 = sbox[y_prime ^ z]
                        if (val1 ^ val2) == dy:
                            found = True
                            break
                    
                    if found:
                        count += 1
                
                bdt[dx][beta][dy] = count
    
    return bdt

def find_incompatibilities():
    """Find all incompatible triples (Δ, β, ∇)"""
    print("=" * 60)
    print("SATURNIN S-BOX INCOMPATIBILITY FINDER")
    print("=" * 60)
    print()
    
    print("Step 4: Searching for incompatibilities...")
    print("Looking for (Δ, β, ∇) where:")
    print("  • DDT[Δ][β] > 0  (differential exists)")
    print("  • BCT[Δ][∇] > 0  (boomerang switch possible)")
    print("  • BDT[Δ][β][∇] = 0  (but actual combination impossible!)")
    print()
    
    incompatibilities = []
    n = len(SATURNIN_SBOX)
    
    for delta in range(1, n):
        for beta in range(1, n):
            for nabla in range(1, n):
                if (ddt[delta][beta] > 0 and 
                    bct[delta][nabla] > 0 and 
                    bdt[delta][beta][nabla] == 0):
                    
                    incompatibilities.append({
                        'delta': delta,
                        'beta': beta,
                        'nabla': nabla,
                        'ddt': ddt[delta][beta],
                        'bct': bct[delta][nabla],
                        'bdt': 0
                    })
    
    print("=" * 60)
    print(f"RESULTS: Found {len(incompatibilities)} incompatibilities!")
    print("=" * 60)
    print()
    
    if incompatibilities:
        for idx, inc in enumerate(incompatibilities[:10], 1):
            print(f"[{idx}] INCOMPATIBILITY FOUND:")
            print(f"  Δ (input diff)      = 0x{inc['delta']:X}")
            print(f"  β (output diff)     = 0x{inc['beta']:X}")
            print(f"  ∇ (boomerang diff)  = 0x{inc['nabla']:X}")
            print(f"  ─────────────────────────────")
            print(f"  DDT[Δ][β] = {inc['ddt']} ✓ (nonzero)")
            print(f"  BCT[Δ][∇] = {inc['bct']} ✓ (nonzero)")
            print(f"  BDT[Δ][β][∇] = {inc['bdt']} ✗ (ZERO - INCOMPATIBLE!)")
            print()
        
        if len(incompatibilities) > 10:
            print(f"... and {len(incompatibilities) - 10} more incompatibilities")
            print()
    else:
        print("No incompatibilities found.")
        print("All BCT/DDT combinations are compatible.")
        print()
    
    print("=" * 60)
    print("Analysis complete!")
    print("=" * 60)
    
    return incompatibilities


print("Step 1: Computing DDT (Differential Distribution Table)...")
ddt = compute_ddt(SATURNIN_SBOX)
print("✓ DDT computed")
print()

print("Step 2: Computing BCT (Boomerang Connectivity Table)...")
bct = compute_bct(SATURNIN_SBOX)
print("✓ BCT computed")
print()

print("Step 3: Computing BDT (Boomerang Difference Table)...")
bdt = compute_bdt(SATURNIN_SBOX)
print("✓ BDT computed")
print()
if __name__ == "__main__":
    incompatibilities = find_incompatibilities()

Step 1: Computing DDT (Differential Distribution Table)...
✓ DDT computed

Step 2: Computing BCT (Boomerang Connectivity Table)...
✓ BCT computed

Step 3: Computing BDT (Boomerang Difference Table)...
✓ BDT computed

SATURNIN S-BOX INCOMPATIBILITY FINDER

Step 4: Searching for incompatibilities...
Looking for (Δ, β, ∇) where:
  • DDT[Δ][β] > 0  (differential exists)
  • BCT[Δ][∇] > 0  (boomerang switch possible)
  • BDT[Δ][β][∇] = 0  (but actual combination impossible!)

RESULTS: Found 822 incompatibilities!

[1] INCOMPATIBILITY FOUND:
  Δ (input diff)      = 0x1
  β (output diff)     = 0x1
  ∇ (boomerang diff)  = 0x2
  ─────────────────────────────
  DDT[Δ][β] = 2 ✓ (nonzero)
  BCT[Δ][∇] = 5 ✓ (nonzero)
  BDT[Δ][β][∇] = 0 ✗ (ZERO - INCOMPATIBLE!)

[2] INCOMPATIBILITY FOUND:
  Δ (input diff)      = 0x1
  β (output diff)     = 0x1
  ∇ (boomerang diff)  = 0x5
  ─────────────────────────────
  DDT[Δ][β] = 2 ✓ (nonzero)
  BCT[Δ][∇] = 5 ✓ (nonzero)
  BDT[Δ][β][∇] = 0 ✗ (ZERO - INCOMPATIBLE!

In [21]:

def get_ddt_pairs(delta, beta):
    """Get all pairs (x, x') that satisfy DDT[delta][beta]"""
    pairs = []
    for x in range(len(SATURNIN_SBOX)):
        x_prime = x ^ delta
        y = SATURNIN_SBOX[x]
        y_prime = SATURNIN_SBOX[x_prime]
        
        if (y ^ y_prime) == beta:
            pairs.append((x, x_prime, y, y_prime))
    
    return pairs

def get_bct_pairs(delta, nabla):
    """Get all pairs (x, x') with their valid z values for BCT[delta][nabla]"""
    pairs = []
    for x in range(len(SATURNIN_SBOX)):
        x_prime = x ^ delta
        y = SATURNIN_SBOX[x]
        y_prime = SATURNIN_SBOX[x_prime]
        
        valid_z = []
        for z in range(len(SATURNIN_SBOX)):
            val1 = SATURNIN_SBOX[y ^ z]
            val2 = SATURNIN_SBOX[y_prime ^ z]
            
            if (val1 ^ val2) == nabla:
                valid_z.append(z)
        
        if valid_z:
            pairs.append((x, x_prime, y, y_prime, valid_z))
    
    return pairs

# The case from your output
delta = 0x1
beta = 0x3
nabla = 0x2

print("DDT PAIRS (Δ=0x1, β=0x3):")
print("-" * 50)
ddt_pairs = get_ddt_pairs(delta, beta)
for x, x_prime, y, y_prime in ddt_pairs:
    print(f"x=0x{x:X}, x'=0x{x_prime:X}, y=0x{y:X}, y'=0x{y_prime:X}")

print(f"\nTotal DDT pairs: {len(ddt_pairs)}")

print("\n" + "=" * 50)
print("\nBCT PAIRS (Δ=0x1, ∇=0x1):")
print("-" * 50)
bct_pairs = get_bct_pairs(delta, nabla)
for x, x_prime, y, y_prime, z_vals in bct_pairs:
    print(f"x=0x{x:X}, x'=0x{x_prime:X}, y=0x{y:X}, y'=0x{y_prime:X}, z={[f'0x{z:X}' for z in z_vals]}")

print(f"\nTotal BCT pairs: {len(bct_pairs)}")

print("\n" + "=" * 50)
print("\nOVERLAP CHECK:")
print("-" * 50)
ddt_x_values = {x for x, _, _, _ in ddt_pairs}
bct_x_values = {x for x, _, _, _, _ in bct_pairs}
overlap = ddt_x_values & bct_x_values

print(f"DDT x values: {sorted([f'0x{x:X}' for x in ddt_x_values])}")
print(f"BCT x values: {sorted([f'0x{x:X}' for x in bct_x_values])}")
print(f"Overlap: {sorted([f'0x{x:X}' for x in overlap])}")

DDT PAIRS (Δ=0x1, β=0x3):
--------------------------------------------------
x=0x6, x'=0x7, y=0x4, y'=0x7
x=0x7, x'=0x6, y=0x7, y'=0x4

Total DDT pairs: 2


BCT PAIRS (Δ=0x1, ∇=0x1):
--------------------------------------------------
x=0x0, x'=0x1, y=0x6, y'=0x9, z=['0x4', '0xB']
x=0x1, x'=0x0, y=0x9, y'=0x6, z=['0x4', '0xB']
x=0x8, x'=0x9, y=0xB, y'=0xD, z=['0xB', '0xD']
x=0x9, x'=0x8, y=0xD, y'=0xB, z=['0xB', '0xD']
x=0xE, x'=0xF, y=0xC, y'=0xA, z=['0xA', '0xC']
x=0xF, x'=0xE, y=0xA, y'=0xC, z=['0xA', '0xC']

Total BCT pairs: 6


OVERLAP CHECK:
--------------------------------------------------
DDT x values: ['0x6', '0x7']
BCT x values: ['0x0', '0x1', '0x8', '0x9', '0xE', '0xF']
Overlap: []


# now finding a compatible trail in 2 rounds

In [31]:
#!/usr/bin/env python3
"""
Find compatible 2-round boomerang trails in Saturnin
"""

# Saturnin S-box (4-bit)
SBOX = SATURNIN_SBOX

INV_SBOX = [0] * 16
for i, v in enumerate(SBOX):
    INV_SBOX[v] = i

def apply_sbox(state):
    """Apply 4-bit S-box to each nibble in a state list."""
    return [SBOX[x & 0xF] for x in state]

def apply_inv_sbox(state):
    """Apply inverse S-box to each nibble in a state list."""
    return [INV_SBOX[x & 0xF] for x in state]

# ---------------------------
# SR_slice for 4x4 layer (per-layer)
# ---------------------------
def SR_slice_layer(s16):
    """ShiftRows on a single 4x4 layer given as 16-list."""
    r = [0] * 16
    # Row 0: no shift
    r[0], r[4], r[8], r[12] = s16[0], s16[4], s16[8], s16[12]
    # Row 1: left shift by 1
    r[1], r[5], r[9], r[13] = s16[5], s16[9], s16[13], s16[1]
    # Row 2: left shift by 2
    r[2], r[6], r[10], r[14] = s16[10], s16[14], s16[2], s16[6]
    # Row 3: left shift by 3
    r[3], r[7], r[11], r[15] = s16[15], s16[3], s16[7], s16[11]
    return r

def SR_slice_layer_inv(s16):
    """Inverse ShiftRows on a single 4x4 layer"""
    r = [0] * 16
    # Row 0
    r[0], r[4], r[8], r[12] = s16[0], s16[4], s16[8], s16[12]
    # Row 1: right shift by 1
    r[1], r[5], r[9], r[13] = s16[13], s16[1], s16[5], s16[9]
    # Row 2: right shift by 2
    r[2], r[6], r[10], r[14] = s16[10], s16[14], s16[2], s16[6]
    # Row 3: right shift by 3
    r[3], r[7], r[11], r[15] = s16[7], s16[11], s16[15], s16[3]
    return r

def SR_slice_3D(state64):
    """Apply SR_slice to each of the 4 layers."""
    out = state64.copy()
    for layer in range(4):
        base = layer * 16
        layer_slice = state64[base:base+16]
        out[base:base+16] = SR_slice_layer(layer_slice)
    return out

def SR_slice_inv_3D(state64):
    out = state64.copy()
    for layer in range(4):
        base = layer * 16
        layer_slice = state64[base:base+16]
        out[base:base+16] = SR_slice_layer_inv(layer_slice)
    return out

# ---------------------------
# MUL and MDS (per-layer)
# ---------------------------
def MUL(t):
    """MUL operation from Saturnin MDS on a 4-element vector"""
    return [t[1], t[2], t[3], (t[0] ^ t[1]) & 0xF]

def MUL_inv(t):
    """Inverse of MUL"""
    return [(t[3] ^ t[0]) & 0xF, t[0] & 0xF, t[1] & 0xF, t[2] & 0xF]

def MDS_layer(s16):
    """Apply Saturnin-like MDS to a 4x4 layer (16 nibbles)"""
    A = s16[0:4]
    B = s16[4:8]
    C = s16[8:12]
    D = s16[12:16]

    C = [(C[i] ^ D[i]) & 0xF for i in range(4)]
    A = [(A[i] ^ B[i]) & 0xF for i in range(4)]
    B = MUL(B)
    D = MUL(D)
    B = [(B[i] ^ C[i]) & 0xF for i in range(4)]
    D = [(D[i] ^ A[i]) & 0xF for i in range(4)]
    A = MUL(MUL(A))
    C = MUL(MUL(C))
    C = [(C[i] ^ D[i]) & 0xF for i in range(4)]
    A = [(A[i] ^ B[i]) & 0xF for i in range(4)]
    B = [(B[i] ^ C[i]) & 0xF for i in range(4)]
    D = [(D[i] ^ A[i]) & 0xF for i in range(4)]

    return A + B + C + D

def MDS_layer_inv(s16):
    """Inverse MDS for a single 4x4 layer"""
    A = s16[0:4]
    B = s16[4:8]
    C = s16[8:12]
    D = s16[12:16]

    B = [(B[i] ^ C[i]) & 0xF for i in range(4)]
    D = [(D[i] ^ A[i]) & 0xF for i in range(4)]
    C = [(C[i] ^ D[i]) & 0xF for i in range(4)]
    A = [(A[i] ^ B[i]) & 0xF for i in range(4)]
    A = MUL_inv(MUL_inv(A))
    C = MUL_inv(MUL_inv(C))
    B = [(B[i] ^ C[i]) & 0xF for i in range(4)]
    D = [(D[i] ^ A[i]) & 0xF for i in range(4)]
    B = MUL_inv(B)
    D = MUL_inv(D)
    C = [(C[i] ^ D[i]) & 0xF for i in range(4)]
    A = [(A[i] ^ B[i]) & 0xF for i in range(4)]

    return A + B + C + D

def MDS_3D(state64):
    out = state64.copy()
    for layer in range(4):
        base = layer * 16
        out[base:base+16] = MDS_layer(state64[base:base+16])
    return out

def MDS_inv_3D(state64):
    out = state64.copy()
    for layer in range(4):
        base = layer * 16
        out[base:base+16] = MDS_layer_inv(state64[base:base+16])
    return out

# Linear layer = MDS + ShiftRows
def linear_layer(state64):
    """Full linear layer: MDS then ShiftRows"""
    state = MDS_3D(state64)
    state = SR_slice_3D(state)
    return state

def linear_layer_inv(state64):
    """Inverse linear layer: Inverse ShiftRows then Inverse MDS"""
    state = SR_slice_inv_3D(state64)
    state = MDS_inv_3D(state)
    return state


# ---------------------------
# Helper: compute difference
# ---------------------------
def xor_states(s1, s2):
    """XOR two states element-wise"""
    return [(s1[i] ^ s2[i]) & 0xF for i in range(len(s1))]

# ---------------------------
# Main trail search
# ---------------------------
def find_compatible_trails():
    print("=" * 70)
    print("SEARCHING FOR COMPATIBLE 2-ROUND BOOMERANG TRAILS IN SATURNIN")
    print("=" * 70)
    print()

    compatible_trails = []
    tested = 0
    
    for pos in range(64):  # test all nibble positions
        for p1_nibble in range(16):
            p1 = [0] * 64
            p1[pos] = p1_nibble

            for initial_diff_nibble in range(1, 16):
                initial_diff = [0] * 64
                initial_diff[pos] = initial_diff_nibble

                p2 = xor_states(p1, initial_diff)

                # --- forward 1.5 rounds ---
                after_sb1 = apply_sbox(p1)
                after_sb2 = apply_sbox(p2)
                c1 = linear_layer(after_sb1)
                c2 = linear_layer(after_sb2)
                c3 = apply_sbox(c2)

                # test all bottom differences (one nibble active)
                # for bottom_pos in range(64):
                bottom_pos = pos
                for bottom_diff_nibble in range(1, 16):
                    bottom_diff = [0] * 64
                    bottom_diff[bottom_pos] = bottom_diff_nibble
                    c4 = xor_states(c3, bottom_diff)

                    # --- backward 1.5 rounds ---
                    c3_inv_sb = apply_inv_sbox(c3)
                    c4_inv_sb = apply_inv_sbox(c4)
                    final_state1 = linear_layer_inv(c3_inv_sb)
                    final_state2 = linear_layer_inv(c4_inv_sb)
                    final_diff = xor_states(final_state1, final_state2)

                    tested += 1

                    # Check BCT for the nibble values (not positions)
                    if bct[initial_diff[pos]][final_diff[pos]] > 0:
                        trail = {
                            'p1_pos': pos,
                            'bottom_pos': bottom_pos,
                            'p1_nibble': p1_nibble,
                            'initial_diff': initial_diff_nibble,
                            'bottom_diff': bottom_diff_nibble,
                            'final_diff': final_diff[pos],
                            'bct_value': bct[initial_diff[pos]][final_diff[pos]],
                            'after_sb1': after_sb1,
                            'after_sb2': after_sb2,
                            'c1': c1,
                            'c2': c2,
                            'c3': c3,
                            'c4': c4,
                            'final_diff_state': final_diff,
                        }
                        compatible_trails.append(trail)

                        # Print full trail details (first few only)
                        if len(compatible_trails) <= 5:
                            print(f"✓ COMPATIBLE TRAIL FOUND")
                            print(f"  Active nibble position (top): {pos}")
                            print(f"  Active nibble position (bottom): {bottom_pos}")
                            print(f"  p1_nibble = 0x{p1_nibble:X}")
                            print(f"  Δ_in (initial_diff) = 0x{initial_diff_nibble:X}")
                            print(f"  Δ_bottom = 0x{bottom_diff_nibble:X}")
                            print(f"  Δ_out (final_diff) = 0x{final_diff[pos]:X}")
                            print(f"  BCT[0x{initial_diff[pos]:X}][0x{final_diff[pos]:X}] = {bct[initial_diff[pos]][final_diff[pos]]}")
                            print("  --- Intermediate states ---")
                            print(f"  after_sb1: {after_sb1}")
                            print(f"  after_sb2: {after_sb2}")
                            print(f"  c1 (after linear): {c1}")
                            print(f"  c2 (after linear): {c2}")
                            print(f"  c3 (after next S-box): {c3}")
                            print(f"  c4 (after bottom diff): {c4}")
                            print(f"  final_diff (after inverse): {final_diff}")
                            print()

    print("=" * 70)
    print(f"RESULTS:")
    print(f"  Tested combinations: {tested}")
    print(f"  Compatible trails found: {len(compatible_trails)}")
    print("=" * 70)
    
    return compatible_trails

if __name__ == "__main__":
    trails = find_compatible_trails()

SEARCHING FOR COMPATIBLE 2-ROUND BOOMERANG TRAILS IN SATURNIN

✓ COMPATIBLE TRAIL FOUND
  Active nibble position (top): 4
  Active nibble position (bottom): 4
  p1_nibble = 0x0
  Δ_in (initial_diff) = 0x1
  Δ_bottom = 0x1
  Δ_out (final_diff) = 0x7
  BCT[0x1][0x7] = 7
  --- Intermediate states ---
  after_sb1: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
  after_sb2: [6, 6, 6, 6, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
  c1 (after linear): [6, 0, 6, 0, 0, 6, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0]
  c2 (after linear): [6, 0, 6, 15, 15, 6, 15, 15, 9, 0, 9, 15, 15, 6, 0, 0,

In [38]:
print(ddt[1][7])
print(ddt[2][7])

2
2


### probablity of the above trail which i found is 
2/16 ki pow 4
1/8 ^ 4
2^-12